In [13]:
# Initialisation and loadup
from http import server
import pandas as pd
import numpy as np
import copy
import os
import time
import re
p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+' #REparser

import selenium as sl
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driverpath=ChromeDriverManager().install()

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [14]:
# initialise the pandas dataframe
columns = ["Index_Link", "Legal_Name", "Town/Suburb","State", "Status", "Size","ABN","Website","Revenue","Expenses","Cause/s","Last_report_date","ref_religion"]

#list off lowercase religous words
religious_buzzwords = ['church','priest','christian','islam', 'muslim','religion','religious','evangelical']

#extend religious word dictionary
CapWords=[]
CAPWORDS=[]
for word in religious_buzzwords:
    CapWords.append(word.capitalize())
    CAPWORDS.append(word.upper())
religious_buzzwords.extend(w for w in CapWords)
religious_buzzwords.extend(w for w in CAPWORDS)

def open_drivers(num_drivers=5):
    drivers =[]
    for i in range(num_drivers):
        drivers.append(webdriver.Chrome(service=Service(driverpath)))
    return drivers

In [85]:
def extract_page_data(d):
    try:
        WebDriverWait(d,35).until(EC.presence_of_element_located((By.XPATH,"//p[contains(text(),': $')]")))
        rev_and_expenses = d.find_elements(By.XPATH,"//p[contains(text(),': $')]")
        revenue = float(re.findall(p, rev_and_expenses[0].text)[0].replace(",",""))
        expenses =float(re.findall(p, rev_and_expenses[1].text)[0].replace(",",""))
    except:
        revenue = None
        expenses = None
        print('revecpenses failed')

    
    # last report date
    try:
        lastreport = d.find_elements(by=By.XPATH, value="//*[contains(text(),'Last reported:')]/..//dd")[0].text
    except:
        lastreport = None
        print("date failed")

    try:
        causes = d.find_elements(by=By.XPATH, value="//*[contains(text(),'Who the charity helps:')]/..//dd")[0].text.replace("\n","; ")
    except:
        causes = None
        print('causes failed')

    try:
        website = d.find_element(by=By.XPATH, value="//*[contains(text(),'Website:')]/..//a").get_attribute('href')
    except:
        website = None
        print("no website/failed")

    religious_ref = 0
    page_text = d.find_element(By.XPATH, '//*').text
    for word in religious_buzzwords:
        if word in page_text:
            religious_ref+=1
            break
        else:
            continue

    
    page_data = [website,revenue,expenses,causes,lastreport,religious_ref]
    return page_data #in form [Website,Revenue,Expenses,Cause/s,Last_report_date,ref_religion]
extract_page_data(testdriver)

['http://www.lifesourceechuca.com/',
 91366.0,
 58604.0,
 'Aboriginal and Torres Strait Islander people; Adults - aged 25 to under 65; Children - aged 6 to under 15; Families; Females; Financially disadvantaged people; General community in Australia; Males; Migrants, refugees or asylum seekers; People at risk of homelessness/ people experiencing homelessness; People from a culturally and linguistically diverse background; People in rural/regional/remote communities; People with chronic illness (including terminal illness); People with disabilities',
 '16 Apr 2022',
 1]

In [89]:
np.load('./s2scrapedata/scrapedfileindex.npy')

array(['S1scrape_topage104.csv'], dtype='<U32')

In [16]:
testdriver = open_drivers(1)[0]

In [84]:
testdriver.get('https://www.acnc.gov.au/charity/charities/e61eb230-39af-e811-a963-000d3ad24077/profile') #religious

In [76]:
testdriver.get('https://www.acnc.gov.au/charity/charities/3e6fb92a-39af-e811-a963-000d3ad24077/profile') #notreligious

In [52]:
extract_page_data(testdriver)

no website/failed
<selenium.webdriver.remote.webelement.WebElement (session="4cfad3849765bfd013b0a3c08fe33741", element="9a031b62-f26d-4096-807f-0c5f9935652c")>


[None,
 91366.0,
 58604.0,
 'Aboriginal and Torres Strait Islander people; Adults - aged 25 to under 65; Children - aged 6 to under 15; Families; Females; Financially disadvantaged people; General community in Australia; Males; Migrants, refugees or asylum seekers; People at risk of homelessness/ people experiencing homelessness; People from a culturally and linguistically diverse background; People in rural/regional/remote communities; People with chronic illness (including terminal illness); People with disabilities',
 '16 Apr 2022',
 1]